In [5]:
import argparse
import numpy as np
import os
import pandas as pd
import tensorflow as tf
from tensorflow import keras

In [41]:
#classes:

#window generator class:
class WindowGenerator:
    def __init__(self, input_width, output_width, mean, std):
        self.input_width = input_width
        self.output_width = output_width
        self.mean = tf.reshape(tf.convert_to_tensor(mean), [1, 1, 2])
        self.std = tf.reshape(tf.convert_to_tensor(std), [1, 1, 2])

    def split_window(self, features):
        inputs = features[:, :-self.output_width, :]

        labels = features[:, -self.output_width, :]

        inputs.set_shape([None,  2])

        labels.set_shape([None, 2])

        return inputs, labels

    def normalize(self, features):
        features = (features - self.mean) / (self.std + 1.e-6)

        return features

    def preprocess(self, features):
        inputs, labels = self.split_window(features)
        inputs = self.normalize(inputs)
        return inputs, labels

    def make_dataset(self, data, train):
        ds = tf.keras.preprocessing.timeseries_dataset_from_array(
                data=data,
                targets=None,
                sequence_length=input_width+output_width,
                sequence_stride=1,
                batch_size=32)
        ds = ds.map(self.preprocess)
        ds = ds.cache()
        if train is True:
            ds = ds.shuffle(100, reshuffle_each_iteration=True)

        return ds


#Multi output class for MAE:
class MultipleOutputforMAE(tf.keras.metrics.Metric):
  def __init__(self, name='MAE', **kwargs):
      super().__init__(name=name, **kwargs)
      self.total = self.add_weight(name='total', initializer='zeros',shape=(2,))
      self.count = self.add_weight('count',initializer = 'zeros')
  def update_state(self, y_true, y_pred, sample_weight=None):
    error = tf.abs(y_pred - y_true)
    error = tf.reduce_mean(error,axis = 0)
    self.total.assign_add(error)
    self.count.assign_add(1.)

  def reset_states(self):
    self.count.assign(tf.zeros_like(self.count))
    self.total.assign(tf.zeros_like(self.total))
    return

  def result(self):
    result = tf.math.divide_no_nan(self.total,self.count)
    return result

In [27]:
#input arguments:
#parser = argparse.ArgumentParser()
#parser.add_argument('--version', type=str, required=True, help='model name')
#parser.add_argument('--number_epochs', type=int, required=False, default=1, help='number of epochs')
#args = parser.parse_args()


In [28]:
version = 'a'
number_epochs = 1

In [29]:

#defining changes for different versions:
if(version == 'a'):
    output_width = 3
    
    #mlp model:
    MLPmodel = keras.Sequential([
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(2)
    ])

    #cnn model:
    CNNmodel = keras.Sequential([
    keras.layers.Conv1D(filters=64, kernel_size=3,activation='relu'),
    keras.layers.Flatten(),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(1)
    ])

    #LSTM model:
    LSTMmodel = keras.Sequential([
    keras.layers.LSTM(units=64),
    keras.layers.Flatten(),
    keras.layers.Dense(1)
    ])

if(version == 'b'):
    output_width = 9

    #mlp model:
    MLPmodel = keras.Sequential([
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(2)
    ])

    #cnn model:
    CNNmodel = keras.Sequential([
    keras.layers.Conv1D(filters=64, kernel_size=3,activation='relu'),
    keras.layers.Flatten(),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(1)
    ])

    #LSTM model:
    LSTMmodel = keras.Sequential([
    keras.layers.LSTM(units=64),
    keras.layers.Flatten(),
    keras.layers.Dense(1)
    ])

In [30]:
#defining seed to have consistent results
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)

In [31]:
#reading the files:
csv_path = 'I:\Polito\ML4IOT\CODES TO DO\Lab3\ex1\jena_climate_2009_2016.csv'
df = pd.read_csv(csv_path)
column_indices = [2, 5]
columns = df.columns[column_indices]
data = df[columns].values.astype(np.float32)

In [32]:
#sperating train,val,test:
n = len(data)
train_data = data[0:int(n*0.7)]
val_data = data[int(n*0.7):int(n*0.9)]
test_data = data[int(n*0.9):]

In [33]:
#defining important variables:
mean = train_data.mean(axis=0)
std = train_data.std(axis=0)
input_width = 6

In [42]:
#make the datasets:
generator = WindowGenerator(input_width, output_width, mean, std)
train_ds = generator.make_dataset(train_data, True)
val_ds = generator.make_dataset(val_data, False)
test_ds = generator.make_dataset(test_data, False)

<dtype: 'float32'>


ValueError: in user code:

    File "C:\Users\imanp\AppData\Local\Temp/ipykernel_21592/2224297020.py", line 28, in preprocess  *
        inputs, labels = self.split_window(features)
    File "C:\Users\imanp\AppData\Local\Temp/ipykernel_21592/3372640436.py", line 18, in split_window  *
        labels.set_shape([None, self.output_width, 2])

    ValueError: Shapes must be equal rank, but are 2 and 3


In [ ]:
#choose the metric system:
metrics = [MultipleOutputforMAE()]

In [ ]:
#compiling models:
MLPmodel.compile(optimizer='adam',
              loss=tf.keras.losses.MAE,
              metrics=metrics)

CNNmodel.compile(optimizer='adam',
              loss=tf.keras.losses.MAE,
              metrics=metrics)

LSTMmodel.compile(optimizer='adam',
              loss=tf.keras.losses.MAE,
              metrics=metrics)



In [ ]:
#running the models:
print("starting MLP")
MLPmodel.fit(train_ds, epochs=number_epochs)
print("MLP done")
print("starting CNNmodel")
CNNmodel.fit(train_ds, epochs=number_epochs)
LSTMmodel.fit(train_ds, epochs=number_epochs)

In [ ]:
test_loss, test_acc = MLPmodel.evaluate(test_ds, verbose=0)
print('\n MSE for MLP :', test_acc)